# Derivation of spherical law of cosines using Sympy and quaternions

## Approach

Start at $w$, move distance $b$, turn by angle $C$, move distance $b$, where am I?

## Computing $u$, $v$ and $w$

![Diagram with u, v and w on Wikipedia](https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Law-of-haversines.svg/330px-Law-of-haversines.svg.png)

In [1]:
import sympy
from sympy import *

Model quaternions as 4x4 matrices

In [2]:
unit = Matrix([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
i = Matrix([[0,1,0,0],[-1,0,0,0],[0,0,0,1],[0,0,-1,0]]).T
j = Matrix([[0,0,1,0],[0,0,0,-1],[-1,0,0,0],[0,1,0,0]]).T
k = i*j

Define $w$

In [3]:
ws_longitude = symbols('ws_longitude')
w = cos(ws_longitude) * i + sin(ws_longitude) * j

Define rotation operator that maps $w$ to $u$

In [4]:
axis_of_rot_from_w_to_u = k
b = symbols('b')
rot_from_w_to_u = cos(b/2) * unit + axis_of_rot_from_w_to_u*sin(b/2)

Apply rotation operator to get $u$

In [5]:
u = rot_from_w_to_u.adjugate() * w * rot_from_w_to_u
u.simplify()
u

Matrix([
[                     0, -cos(b - ws_longitude), sin(b - ws_longitude),                      0],
[ cos(b - ws_longitude),                      0,                     0, -sin(b - ws_longitude)],
[-sin(b - ws_longitude),                      0,                     0, -cos(b - ws_longitude)],
[                     0,  sin(b - ws_longitude), cos(b - ws_longitude),                      0]])

Rotate the rotation axis of $WU$ by angle C around $u$ to get the rotation axis of $UV$

In [6]:
C = symbols('C')
axis_of_rot_from_u_to_v = (cos(C/2) * unit - sin(C/2) * u) * axis_of_rot_from_w_to_u * (cos(C/2) * unit + sin(C/2) * u)
axis_of_rot_from_u_to_v.simplify()
axis_of_rot_from_u_to_v

Matrix([
[                                                        0, -cos(C - b + ws_longitude)/2 + cos(C + b - ws_longitude)/2, -sin(C - b + ws_longitude)/2 - sin(C + b - ws_longitude)/2,                                                    -cos(C)],
[cos(C - b + ws_longitude)/2 - cos(C + b - ws_longitude)/2,                                                          0,                                                    -cos(C),  sin(C - b + ws_longitude)/2 + sin(C + b - ws_longitude)/2],
[sin(C - b + ws_longitude)/2 + sin(C + b - ws_longitude)/2,                                                     cos(C),                                                          0, -cos(C - b + ws_longitude)/2 + cos(C + b - ws_longitude)/2],
[                                                   cos(C), -sin(C - b + ws_longitude)/2 - sin(C + b - ws_longitude)/2,  cos(C - b + ws_longitude)/2 - cos(C + b - ws_longitude)/2,                                                          0]])

Define rotation operator that maps $u$ to $v$

In [7]:
a = symbols('a')
rot_from_u_to_v = cos(a/2) * unit + axis_of_rot_from_u_to_v * sin(a/2)
rot_from_u_to_v.simplify()
# Compute determinant to check if it's 1
det(rot_from_u_to_v).simplify()

1

Apply rotation operator to get $v$

In [8]:
v = rot_from_u_to_v.adjugate() * u * rot_from_u_to_v
v.simplify()
v[:,0]

Matrix([
[                                                                  0],
[-sin(a)*sin(b - ws_longitude)*cos(C) + cos(a)*cos(b - ws_longitude)],
[-sin(a)*cos(C)*cos(b - ws_longitude) - sin(b - ws_longitude)*cos(a)],
[                                        cos(C - a)/2 - cos(C + a)/2]])

## Analysing sides of triangle (**proving the law of cosines**)

Notice below that $\cos(c) = -\sin(a) \sin(b) \cos(C) + \cos(a) \cos(b)$. **This proves the law of cosines**

In [9]:
wv_side = v * w.adjugate()
wv_side.simplify()
wv_side[:,0]

Matrix([
[-sin(a)*sin(b)*cos(C) + cos(a)*cos(b)],
[      sin(C)*sin(a)*sin(ws_longitude)],
[     -sin(C)*sin(a)*cos(ws_longitude)],
[-sin(a)*cos(C)*cos(b) - sin(b)*cos(a)]])

## Analysing angles between sides

Observe that $\cos A = \frac{1}{4}(2\cos(a - 2b) + 2\cos(a + 2b) + \cos(-C + a + 2b) - \cos(C - a + 2b) - \cos(C + a - 2b) + \cos(C + a + 2b))$

In [10]:
wu_side = w * u.adjugate()
wu_side.simplify()

rotation_from_wu_to_wv = wu_side * wv_side.adjugate()
rotation_from_wu_to_wv.simplify()
rotation_from_wu_to_wv[:,0]

Matrix([
[ cos(a - 2*b)/2 + cos(a + 2*b)/2 + cos(-C + a + 2*b)/4 - cos(C - a + 2*b)/4 - cos(C + a - 2*b)/4 + cos(C + a + 2*b)/4],
[                                                                                 -sin(C)*sin(a)*sin(b + ws_longitude)],
[                                                                                  sin(C)*sin(a)*cos(b + ws_longitude)],
[-sin(a - 2*b)/2 + sin(a + 2*b)/2 + sin(-C + a + 2*b)/4 - sin(C - a + 2*b)/4 + sin(C + a - 2*b)/4 + sin(C + a + 2*b)/4]])

Which we can make Sympy express as $\cos A = -\sin(a)\sin(2b)\cos(C) + \cos(a)\cos(2b)$

In [11]:
rotation_from_wu_to_wv[0,0].rewrite(exp).simplify().expand().rewrite(cos).expand()

-sin(a)*sin(2*b)*cos(C) + cos(a)*cos(2*b)